# Geocoding raw address data for the Who Owns Hoboken? app

This script utilizes the Nominatim API and geopy to append latitude and longitude to the raw data compiled for the Who Owns Hoboken? app.

In [45]:
## Import libraries
import pandas as pd
import numpy as np
from geopy import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from pandas_geojson import to_geojson
from pandas_geojson import write_geojson
pd.options.mode.chained_assignment = None

In [292]:
## Read data
data = pd.read_csv('./raw_data.csv')
data.head()

,Block,Lot,Qual,Class,Property Location,Building name,LLC/Owner,LLC or private individual,Total Units,Company,...,Building Class,Prior Block,Prior Lot,Prior Qual,Updated,Additional Lots,Rent Control,Building Desc,units2,Sale Date
0,75.0,1.0,C004D,2,601 MONROE ST,NaN,"OCONNOR, JAMES P JR",Individual,2.0,James O'Connor,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,2BED,1.0,0000-00-00
1,219.0,8.0,C0002,2,815 WASHINGTON ST,NaN,"OCONNOR, JAMES & BERNADETTE",Individual,2.0,James O'Connor,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,2BR,1.0,8/8/11
2,207.0,25.0,NaN,2,808 WASHINGTON ST,NaN,808 WASHINGTON ST LLC,LLC,4.0,NaN,...,49.0,NaN,NaN,NaN,12/27/19,NaN,YES,4B-4U-H-BA,4.0,8/16/13
3,180.0,6.0,NaN,2,511 PARK AVE,NaN,"RAD, MOHAMMAD TRUSTEE OF JOODI",Trust,2.0,Rad,...,49.0,NaN,NaN,NaN,12/27/19,NaN,YES,3B-2U-FX-H,2.0,5/12/14
4,88.0,1.0,NaN,4C,800 MADISON/801 MONROE ST,Avalon,DSF IV HOBOKEN OWNER LLC % AVALON B,LLC,220.0,AvalonBay,...,NaN,NaN,NaN,NaN,NaN,NaN,YES,6B-220U-4C-G,220.0,NaN


In [74]:
data.columns = data.columns.map(lambda x : x.replace('.', '').replace(' ', '_').replace('/','_').replace("'", '')) 
data.columns = data.columns.str.lower()

In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2773 entries, 0 to 2772
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   block                      2773 non-null   float64
 1   lot                        2773 non-null   float64
 2   qual                       1067 non-null   object 
 3   property_location          2773 non-null   object 
 4   building_name              60 non-null     object 
 5   llc_owner                  2773 non-null   object 
 6   llc_or_private_individual  2773 non-null   object 
 7   total_units                2772 non-null   float64
 8   company                    1012 non-null   object 
 9   owners_mailing_address     2773 non-null   object 
 10  city_state_zip             2773 non-null   object 
 11  sq_ft                      2411 non-null   float64
 12  yr_built                   2020 non-null   float64
 13  rent_control               2773 non-null   objec

In [294]:
## Make address all uppercase
data['Property Location'] = data['Property Location'].str.upper()

In [295]:
## Clean up street names so geocoder can recognize them 
data['Property Location'] = data['Property Location'].str.replace('FIRST', '1ST')
data['Property Location'] = data['Property Location'].str.replace('SECOND', '2ND')
data['Property Location'] = data['Property Location'].str.replace('THIRD', '3RD')
data['Property Location'] = data['Property Location'].str.replace('FOURTH ', '4TH ')
data['Property Location'] = data['Property Location'].str.replace('FIFTH', '5TH')
data['Property Location'] = data['Property Location'].str.replace('SIXTH', '6TH')
data['Property Location'] = data['Property Location'].str.replace('SEVENTH', '7TH')
data['Property Location'] = data['Property Location'].str.replace('EIGHTH', '8TH')
data['Property Location'] = data['Property Location'].str.replace('EIGHT ', '8TH ')
data['Property Location'] = data['Property Location'].str.replace('NINTH', '9TH')
data['Property Location'] = data['Property Location'].str.replace('TENTH', '10TH')
data['Property Location'] = data['Property Location'].str.replace('ELEVENTH', '11TH')
data['Property Location'] = data['Property Location'].str.replace('TWELFTH', '12TH')
data['Property Location'] = data['Property Location'].str.replace('THIRTEENTH', '13TH')
data['Property Location'] = data['Property Location'].str.replace('FOURTEENTH', '14TH')
data['Property Location'] = data['Property Location'].str.replace('FIFTEENTH', '15TH')
data['Property Location'] = data['Property Location'].str.replace('WASH ', 'WASHINGTON')
data['Property Location'] = data['Property Location'].str.replace('WASHINGTONST ', 'WASHINGTON ST')
data['Property Location'] = data['Property Location'].str.replace('0BSERVER ', 'OBSERVER')
data['Property Location'] = data['Property Location'].str.replace('OBSERVERHWY ', 'OBSERVER HWY')
data['Property Location'] = data['Property Location'].str.replace('OBSERVERHWY', 'OBSERVER HWY')
data['Property Location'] = data['Property Location'].str.replace('THS T ', 'TH ST')
data['Property Location'] = data['Property Location'].str.replace('ST REAR', 'ST')
data['Property Location'] = data['Property Location'].str.replace('BLOOM ', 'BLOOMFIELD')
data['Property Location'] = data['Property Location'].str.replace('THS T', 'TH ST')
data['Property Location'] = data['Property Location'].str.replace(' AT', ' ST')
data['Property Location'] = data['Property Location'].str.replace('WASHINGTONST', 'WASHINGTON ST')
data['Property Location'] = data['Property Location'].str.replace('ST230', 'ST')

In [296]:
## Clean up addresses with notes in them
sep = '('
data['Property Location'] = data['Property Location'].str.split(sep).str[0]

In [297]:
## Clean up addresses with two locations combined
sep = '/'
data['clean_address'] = data['Property Location'].str.split(sep).str[0]

In [298]:
## Clean up addresses named with intersection
sep = '&'
data['clean_address'] = data['clean_address'].str.split(sep).str[0]

In [299]:
## Create column to be used for address lookup in Nominatim API
data['search_address'] = data['clean_address'] + ', Hoboken, NJ, 07030'
data.head()

,Block,Lot,Qual,Class,Property Location,Building name,LLC/Owner,LLC or private individual,Total Units,Company,...,Prior Lot,Prior Qual,Updated,Additional Lots,Rent Control,Building Desc,units2,Sale Date,clean_address,search_address
0,75.0,1.0,C004D,2,601 MONROE ST,NaN,"OCONNOR, JAMES P JR",Individual,2.0,James O'Connor,...,NaN,NaN,NaN,NaN,YES,2BED,1.0,0000-00-00,601 MONROE ST,"601 MONROE ST, Hoboken, NJ, 07030"
1,219.0,8.0,C0002,2,815 WASHINGTON ST,NaN,"OCONNOR, JAMES & BERNADETTE",Individual,2.0,James O'Connor,...,NaN,NaN,NaN,NaN,YES,2BR,1.0,8/8/11,815 WASHINGTON ST,"815 WASHINGTON ST, Hoboken, NJ, 07030"
2,207.0,25.0,NaN,2,808 WASHINGTON ST,NaN,808 WASHINGTON ST LLC,LLC,4.0,NaN,...,NaN,NaN,12/27/19,NaN,YES,4B-4U-H-BA,4.0,8/16/13,808 WASHINGTON ST,"808 WASHINGTON ST, Hoboken, NJ, 07030"
3,180.0,6.0,NaN,2,511 PARK AVE,NaN,"RAD, MOHAMMAD TRUSTEE OF JOODI",Trust,2.0,Rad,...,NaN,NaN,12/27/19,NaN,YES,3B-2U-FX-H,2.0,5/12/14,511 PARK AVE,"511 PARK AVE, Hoboken, NJ, 07030"
4,88.0,1.0,NaN,4C,800 MADISON/801 MONROE ST,Avalon,DSF IV HOBOKEN OWNER LLC % AVALON B,LLC,220.0,AvalonBay,...,NaN,NaN,NaN,NaN,YES,6B-220U-4C-G,220.0,NaN,800 MADISON,"800 MADISON, Hoboken, NJ, 07030"


In [300]:
## Define Nominatim API instance
locator = Nominatim(user_agent='who_owns_hoboken')

In [301]:
## Call API for each row of dataset
## Store output from API call in 'location column'
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
data['location'] = data['search_address'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('910 WASHINGTON ST, Hoboken, NJ, 07030',), **{}).
Traceback (most recent call last):
  File "/Users/natehutchinson/opt/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/Users/natehutchinson/opt/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/Users/natehutchinson/opt/anaconda3/lib/python3.9/socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/natehutchinson/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_re

In [302]:
## Extract lat/long information from API output
data['point'] = data['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [303]:
## Turn lat/long information into new columns in the dataset
data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(data['point'].tolist(), index=data.index)

In [304]:
data.groupby('latitude').size().sort_values(ascending = False)

latitude
40.750362    59
40.749529    40
40.752481    34
40.746050    31
40.752710    25
             ..
40.742083     1
40.742080     1
40.742043     1
40.742034     1
40.754934     1
Length: 2064, dtype: int64

In [77]:
## Manually geocode these addresses
data[data['latitude'].isnull()]['property_location'].unique()

array([], dtype=object)

In [316]:
# 2 Constitution Court
data.loc[data['Property Location'] == '2 CONSTITUTION COURT', 'latitude'] = 40.75235433173578
data.loc[data['Property Location'] == '2 CONSTITUTION COURT', 'longitude'] = -74.02416413263948
# 1 Independence Court
data.loc[data['Property Location'] == '1 INDEPENDENCE COURT', 'latitude'] = 40.7517113622344
data.loc[data['Property Location'] == '1 INDEPENDENCE COURT', 'longitude'] = -74.0243229423285
# '1200-1222 HUDSON ST'
data.loc[data['Property Location'] == '1200-1222 HUDSON ST', 'latitude'] = 40.751547793135835
data.loc[data['Property Location'] == '1200-1222 HUDSON ST', 'longitude'] = -74.02566382893585
# '1201-1333 HUDSON ST'
data.loc[data['Property Location'] == '1201-1333 HUDSON ST', 'latitude'] = 40.751327435663356
data.loc[data['Property Location'] == '1201-1333 HUDSON ST', 'longitude'] = -74.02529360594718
# '1201-21R WASHINGTON ST'
data.loc[data['Property Location'] == '1201-21R WASHINGTON ST', 'latitude'] = 40.75167892656019
data.loc[data['Property Location'] == '1201-21R WASHINGTON ST', 'longitude'] = -74.02620949828147
# '224-232 RIVER 235 HUDSON'
data.loc[data['Property Location'] == '224-232 RIVER 235 HUDSON', 'latitude'] = 40.7394286430441
data.loc[data['Property Location'] == '224-232 RIVER 235 HUDSON', 'longitude'] = -74.0286408719967
# '401-03 WASHINGTON ST'
data.loc[data['Property Location'] == '401-03 WASHINGTON ST', 'latitude'] = 40.74134653174973
data.loc[data['Property Location'] == '401-03 WASHINGTON ST', 'longitude'] = -74.02950291302936
# '300-08 RIVER ST'
data.loc[data['Property Location'] == '300-08 RIVER ST', 'latitude'] = 40.74002433251577
data.loc[data['Property Location'] == '300-08 RIVER ST', 'longitude'] = -74.02844601055078
# '11 ELYSIAN PLACE'
data.loc[data['Property Location'] == '11 ELYSIAN PLACE', 'latitude'] = 40.747902005692055
data.loc[data['Property Location'] == '11 ELYSIAN PLACE', 'longitude'] = -74.0248531253788

In [78]:
## add column for what year the building falls under rent control
data.loc[data['yr_built'] == 0, 'yr_built'] = np.nan
data['yr_built'] = data['yr_built'].astype('Int64')
data['rent_control_year'] = data['yr_built'] + 30

In [79]:
## convert units columns to integers
data['total_units'] = data['total_units'].astype('Int64')
data['units2'] = data['units2'].astype('Int64')

In [381]:
## save to csv before splitting into condo/non-condo
data.to_csv('./data/hoboken_landlords.csv')

In [65]:
#with open('./data/hoboken_landlords.csv') as x:
    #ncols = len(x.readline().split(','))

#data = pd.read_csv('./data/hoboken_landlords.csv', usecols=range(1,ncols))

In [5]:
## drop unneccesary columns
drop_cols = ['Class', 'Building Class', 'Prior Block', 'Prior Lot', 'Prior Qual', 'Updated',
       'Additional Lots', 'search_address', 'location', 'point', 'altitude']
data = data.drop(drop_cols, axis = 1)

KeyError: "['Class', 'Building Class', 'Prior Block', 'Prior Lot', 'Prior Qual', 'Updated', 'Additional Lots', 'search_address', 'location', 'point', 'altitude'] not found in axis"

In [81]:
## Change rent control column to binary
data.loc[data['rent_control'] != 'YES', 'rent_control'] = 'NO'

In [80]:
## Make Company = Owner when Company is blank
data.loc[data['company'].isna(), 'company'] = data['llc_owner']

In [82]:
## Replace NAs with 'None'- required for geojson conversion
data = data.fillna(np.nan).replace([np.nan], [None])

In [83]:
## 28% of units are rent controlled
## this may be a slight undercount as I think the data is missing some condo units from RC buildings
data.groupby('rent_control')['units2'].sum() 

rent_control
NO     11766
YES     4563
Name: units2, dtype: object

In [84]:
## 56% of buildings are rent controlled
data.groupby('rent_control').size()

rent_control
NO     1207
YES    1566
dtype: int64

In [85]:
## split data into condos and non-condos
condos = data[data['qual'].str.startswith('C') == True]
apts = data[data['qual'].str.startswith('C') != True]

In [86]:
## define properties for geojson
properties = list(data.columns)
properties.remove('latitude')
properties.remove('longitude')
properties

['block',
 'lot',
 'qual',
 'property_location',
 'building_name',
 'llc_owner',
 'llc_or_private_individual',
 'total_units',
 'company',
 'owners_mailing_address',
 'city_state_zip',
 'sq_ft',
 'yr_built',
 'rent_control',
 'building_desc',
 'units2',
 'sale_date',
 'clean_address',
 'rent_control_year']

In [87]:
## convert data to geojson
condos_geojson = to_geojson(df=condos, lat='latitude', lon='longitude',
                 properties=properties)

apts_geojson = to_geojson(df=apts, lat='latitude', lon='longitude',
                 properties=properties)

In [88]:
## save geojson files in data folder
write_geojson(condos_geojson, filename='./data/condos.geojson', indent=4)
write_geojson(apts_geojson, filename='./data/apts.geojson', indent=4)